In [ ]:
! head -n 4 /kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv

In [ ]:
from random import random
import pandas as pd

trainer = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

toxic = trainer[["more_toxic"]]
toxic["text"] = toxic["more_toxic"]
toxic["label"] = 0.85 + random() / 8 - 0.08

nontoxic = trainer[["less_toxic"]]
nontoxic["text"] = nontoxic["less_toxic"]
nontoxic["label"] = 0.35 + random() / 8 - 0.08

combined = toxic.append(nontoxic).sample(frac=1, random_state=101)
combined = combined[["text", "label"]]
combined["label"] = combined["label"].astype('float')
combined.head(2)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(combined, random_state=101, train_size=0.99)

In [ ]:
! export TOKENIZERS_PARALLELISM=false

In [ ]:
# if you have internet, you can just do: pip install simpletransformers
# thanks to https://www.kaggle.com/code/hengzheng/simpletransformers-regression-starter-less-code
! pip install -q /kaggle/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
! pip install -q simpletransformers==0.51.0 --no-index --find-links=file:///kaggle/input/simpletransformers/simpletransformers-0.51.0

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.max_seq_length = 300
model_args.num_train_epochs = 1
model_args.regression = True
model_args.no_save = True
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model = ClassificationModel(
    "bert",
    "../input/huggingface-bert/bert-large-uncased",
    num_labels=1,
    args=model_args
)

model.train_model(train)

In [ ]:
predictions, raw_outputs = model.predict(test['text'].tolist())
predictions[:10]

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(test['label'].tolist(), predictions)

In [ ]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv

In [ ]:
import csv

texts = []
ids = []
with open('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv', 'r') as rfile:
  rdr = csv.reader(rfile)
  batch = None
  for row in rdr:
    if batch is None:
        batch = True
        continue
    ids.append(row[0])
    texts.append(row[1].replace('"', '').strip())

In [ ]:
sub_predictions, sub_outputs = model.predict(texts)

In [ ]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv

In [ ]:
op = []
id_idx = 0

with open('submission.csv', 'w') as opfile:
  writer = csv.writer(opfile)
  writer.writerow(["comment_id","score"])

  for row in sub_predictions:
      writer.writerow([
        ids[id_idx],
        sub_predictions[id_idx]
      ])
      id_idx += 1

In [ ]:
! head -n 5 submission.csv